# 卷積神經網路

In [25]:
import numpy as np

In [26]:
# 參數定義：
# dz：從下一層傳回的輸出梯度，形狀為 (o_n,)，其中 o_n 是輸出長度
# x：輸入數據，形狀為 (n,)，其中 n 是輸入長度
# w：卷積核/濾波器，形狀為 (K,)，其中 K 是卷積核長度
# pad：Padding 大小，默認為 0
# s：步幅，默認為 1
def conv_backward(dz, x, w, pad=0, s=1):
    # n：輸入長度，K：濾波器/卷積核（Filter）長度
    n, K = len(x), len(w)

    # o_n (輸出長度)：控制迴圈次數，確保視窗掃描範圍正確
    o_n = 1 + (n + 2 * pad - K) // s

    assert(o_n == len(dz))    
    
    # dx (輸入梯度)：準備傳回前一層的梯度；dw (權重梯度)：濾波器/卷積核更新的依據
    dx = np.zeros_like(x)
    dw = np.zeros_like(w)

    # db (Bias 梯度)：輸出梯度 dz 的加總，dz 中每個元素對應輸出的一個位置，對應到每個卷積核的偏置更新
    db = dz[:].sum()
    
    # 進行 Padding 處理與初始化對應梯度空間
    x_pad = np.pad(x, [(pad, pad)], 'constant')
    dx_pad = np.zeros_like(x_pad)
        
    for i in range(o_n):
        start = i * s

        # dw (權重梯度)：由「輸入視窗」與「dz」的乘積累加求得
        dw += x_pad[start:start+K] * dz[i]

        # dx_pad (輸入梯度)：將權重與 dz 相乘，回傳至對應的輸入位置
        dx_pad[start:start+K] += w * dz[i]

    # 移除 Padding 取回原始輸入大小的 dx
    dx = dx_pad[pad:-pad] if pad > 0 else dx_pad

    return dx, dw, db

In [27]:
import numpy as np

# 固定亂數種子，確保實驗結果具備可重複性（Reproducibility）
np.random.seed(231)

# x (輸入資料)：長度為 5 的向量
x = np.random.randn(5)

# w (濾波器)：長度為 3 的權重向量
w = np.random.randn(3)

# s (步幅) 與 pad (填充)：定義卷積滑動的規律與邊界擴充
stride = 2
pad = 1

# dz (下層梯度)：反向傳播的起點，代表「Loss 對卷積輸出的變化率」
# 這裡長度設為 5，是為了配合 pad=1, s=1 的卷積輸出長度
dz = np.random.randn(5)

print("Upstream Gradient (dz):")
print(dz)

# 執行反向傳播計算
# 這裡傳入 s=1，代表以步幅 1 進行梯度回傳計算
dx, dw, db = conv_backward(dz, x, w, pad, 1)

# dx (輸入梯度)：用來更新前一層神經元（或是上一層卷積層）的數值
print("\nInput Gradient (dx):")
print(dx)

# dw (權重梯度)：模型訓練最核心的部分，優化器（Optimizer）會根據它來調整 w
print("\nWeight Gradient (dw):")
print(dw)

# db (Bias 梯度)：Loss 對偏差項的變化率
print("\nBias Gradient (db):")
print(db)

Upstream Gradient (dz):
[-1.4255293  -0.3763567  -0.34227539  0.29490764 -0.83732373]

Input Gradient (dx):
[ 0.50522405 -2.33230266 -0.87796042 -0.03246064  0.67446745]

Weight Gradient (dw):
[-0.56864738 -0.65679696 -1.09889311]

Bias Gradient (db):
-2.6865774833459612


In [28]:
import numpy as np
import math
from modules import init_weights as init_weights

# Layer 的基底類別，其他層都會繼承
# 定義了基本的接口（forward、backward、正則化相關方法），具體實作由子類別完成
class Layer:
    def __init__(self):
        self.params = None  # 存放該層的參數
        pass

    def forward(self, x):
        # forward 尚未實作
        raise NotImplementedError

    def backward(self, x, grad):
        # backward 尚未實作
        raise NotImplementedError

    def reg_grad(self, reg):
        # 正則化梯度（預設不做）
        pass

    def reg_loss(self, reg):
        # 正則化 loss（預設為 0）
        return 0.

    def reg_loss_grad(self, reg):
        # 正則化 loss 與梯度（預設為 0）
        return 0


# 卷積層：實作卷積層的 forward 與 backward，並包含權重初始化與正則化功能
# 參數定義：
# in_channels：輸入的 channel 數量（例如 RGB 圖片為 3）
# out_channels：輸出的 channel 數量（即 filter 的數量）
# kernel_size：卷積核的大小（假設為正方形，則為邊長）
# stride：卷積的步幅，控制卷積核每次移動的距離
# padding：卷積前對輸入進行的零填充大小
class Conv(Layer):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
            super().__init__()
            self.C = in_channels   # 輸入 channel 數
            self.F = out_channels  # filter 數
            self.K = kernel_size   # kernel 大小
            self.S = stride        # stride
            self.P = padding       # padding

            # 卷積權重 (F, C, K, K)
            self.W = np.random.randn(self.F, self.C, self.K, self.K)

            # 每個 filter 一個 bias
            self.b = np.random.randn(self.F,)

            # 參數與梯度列表
            self.params = [self.W, self.b]
            self.grads = [np.zeros_like(self.W), np.zeros_like(self.b)]

            self.X = None  # forward 時記錄輸入
            self.reset_parameters()

    def reset_parameters(self):
        # 使用 Kaiming 初始化權重
        init_weights.kaiming_uniform(self.W, a=math.sqrt(5))

        # 初始化 bias
        if self.b is not None:
            fan_in = self.C
            bound = 1 / math.sqrt(fan_in)
            self.b[:] = np.random.uniform(-bound, bound, self.b.shape)

    def forward(self, X):
        # 記錄輸入資料
        self.X = X

        # 取得輸入與權重大小
        N, C, X_h, X_w = self.X.shape
        F, _, F_h, F_w = self.W.shape

        # 對輸入做 padding
        X_pad = np.pad(
            self.X,
            ((0, 0), (0, 0), (self.P, self.P), (self.P, self.P)),
            mode='constant',
            constant_values=0
        )

        # 計算輸出大小
        O_h = 1 + int((X_h + 2 * self.P - F_h) / self.S)
        O_w = 1 + int((X_w + 2 * self.P - F_w) / self.S)

        # 建立輸出張量
        O = np.zeros((N, F, O_h, O_w))

        # 逐位置做卷積運算
        for n in range(N): # 遍歷每個輸入樣本
            for f in range(F): # 遍歷每個 filter
                for i in range(O_h): # 遍歷輸出高度位置
                    hs = i * self.S # 計算對應的輸入高度位置
                    for j in range(O_w): # 遍歷輸出寬度位置
                        ws = j * self.S # 計算對應的輸入寬度位置
                        # 進行卷積運算：將輸入視窗與 filter 權重相乘並加上 bias，得到輸出位置的值
                        O[n, f, i, j] = (X_pad[n, :, hs:hs+F_h, ws:ws+F_w] * self.W[f]).sum() + self.b[f]

        return O

    def __call__(self, X):
        # 讓 Conv 物件可以直接呼叫
        return self.forward(X)

    def backward(self, dZ):
        # 取得輸出梯度與輸入大小
        N, F, Z_h, Z_w = dZ.shape
        N, C, X_h, X_w = self.X.shape
        F, _, F_h, F_w = self.W.shape

        pad = self.P

        # 計算輸出空間大小
        H_ = 1 + (X_h + 2 * pad - F_h) // self.S
        W_ = 1 + (X_w + 2 * pad - F_w) // self.S

        # 初始化梯度
        dX = np.zeros_like(self.X)
        dW = np.zeros_like(self.W)
        db = np.zeros_like(self.b)

        # padded 的輸入與梯度
        X_pad = np.pad(self.X, [(0,0), (0,0), (pad,pad), (pad,pad)], 'constant')
        dX_pad = np.pad(dX, [(0,0), (0,0), (pad,pad), (pad,pad)], 'constant')

        # 逐位置回傳梯度
        for n in range(N): # 遍歷每個輸入樣本
            for f in range(F): # 遍歷每個 filter
                db[f] += dZ[n, f].sum() # bias 梯度為對應 filter 的輸出梯度總和
                for i in range(H_): # 遍歷輸出高度位置
                    hs = i * self.S # 計算對應的輸入高度位置
                    for j in range(W_): # 遍歷輸出寬度位置
                        ws = j * self.S # 計算對應的輸入寬度位置
                        # 進行卷積運算的反向傳播：更新權重梯度與輸入梯度
                        dW[f] += X_pad[n, :, hs:hs+F_h, ws:ws+F_w] * dZ[n, f, i, j]
                        # 輸入梯度由權重與輸出梯度的乘積累加求得，回傳至對應的輸入位置
                        dX_pad[n, :, hs:hs+F_h, ws:ws+F_w] += self.W[f] * dZ[n, f, i, j]

        # 移除 padding
        dX = dX_pad[:, :, pad:pad+X_h, pad:pad+X_h]

        # 累加梯度
        self.grads[0] += dW
        self.grads[1] += db

        return dX

    def reg_grad(self, reg):
        # L2 正則化梯度
        self.grads[0] += 2 * reg * self.W

    def reg_loss(self, reg):
        # L2 正則化 loss
        return reg * np.sum(self.W**2)

    def reg_loss_grad(self, reg):
        # 同時計算正則化 loss 與梯度
        self.grads[0] += 2 * reg * self.W
        return reg * np.sum(self.W**2)


In [29]:
X_h = 5
X_w = 5
P = 1
F_h = 3
F_w = 3
S = 1
O_h = 1 + int((X_h + 2 * P - F_h) / S)
O_w = 1 + int((X_w + 2 * P - F_w) / S)
print("Output Height (O_h):", O_h)
print("Output Width (O_w):", O_w)

Output Height (O_h): 5
Output Width (O_w): 5


In [30]:
# 設定隨機數種子，確保每次執行產生的隨機數都一樣，方便除錯與結果復現
np.random.seed(1)

# x (輸入資料)：維度分別代表 (Batch Size, Channels, Height, Width)
# 這裡有 4 筆資料、3 個通道、大小為 5x5
x = np.random.randn(4, 3, 5, 5)

print(x.shape)       # 印出輸入維度，確認資料格式是否正確
print(x[0,0],"\n")   # 觀察第 1筆資料在第 1 個通道的數值，確認資料內容是否合理

# 初始化卷積層：輸入 3 通道、輸出 2 通道、Filter 3x3、步幅(Stride) 1、填充(Padding) 1
conv = Conv(3, 2, 3, 1, 1)

# f (輸出特徵圖)：執行前向傳播計算結果
f = conv.forward(x)

print(f.shape)       # 印出輸出維度，確認 Padding 與 Stride 計算是否正確
print(f[0,0],"\n")   # 觀察第 1 筆資料在第 1 個輸出通道的特徵數值

# df (輸出梯度)：模擬從下一層傳回來的梯度 (Upstream Gradient)
# 其形狀必須與前向傳播的輸出 f 完全一致
df = np.random.randn(4, 2, 5, 5)

# dx (輸入梯度)：執行反向傳播，計算要傳回前一層的梯度
dx = conv.backward(df)

print(df[0,0],"\n")  # 印出模擬的原始輸出梯度
print(dx[0,0],"\n")  # 印出計算後對輸入 x 的梯度

# conv.grads (參數梯度)：儲存了此層需要被更新的權重與偏差梯度
print(conv.grads[0][0,0],"\n") # 印出濾波器權重的梯度 (dw)
print(conv.grads[1],"\n")      # 印出偏差的梯度 (db)

(4, 3, 5, 5)
[[ 1.62434536 -0.61175641 -0.52817175 -1.07296862  0.86540763]
 [-2.3015387   1.74481176 -0.7612069   0.3190391  -0.24937038]
 [ 1.46210794 -2.06014071 -0.3224172  -0.38405435  1.13376944]
 [-1.09989127 -0.17242821 -0.87785842  0.04221375  0.58281521]
 [-1.10061918  1.14472371  0.90159072  0.50249434  0.90085595]] 

(4, 2, 5, 5)
[[ 0.46362714 -0.83578144  0.40298519 -0.32152652  0.56616046]
 [-0.47878018  1.02346756  0.20004975  0.59663092  0.25253169]
 [-0.39733747 -0.08368194  0.52454712  0.54133918 -0.32698456]
 [ 0.47703053 -0.01967369  1.13655418  0.22321357  0.77693417]
 [-0.23944267  0.62971182 -0.38411731  0.42818679 -0.07566246]] 

[[-1.30653407  0.07638048  0.36723181  1.23289919 -0.42285696]
 [ 0.08646441 -2.14246673 -0.83016886  0.45161595  1.10417433]
 [-0.28173627  2.05635552  1.76024923 -0.06065249 -2.413503  ]
 [-1.77756638 -0.77785883  1.11584111  0.31027229 -2.09424782]
 [-0.22876583  1.61336137 -0.37480469 -0.74996962  2.0546241 ]] 

[[-1.28063939e-02 -3

In [31]:
def numerical_gradient_from_df(f, p, df, h=1e-5):
  # 建立一個與 p 形狀相同、內容全為 0 的陣列，用來存每個參數的梯度
  grad = np.zeros_like(p)

  # 使用 nditer 逐一走訪 p 中的每一個元素（支援多維陣列）
  it = np.nditer(p, flags=['multi_index'], op_flags=['readwrite'])

  # 只要 iterator 還沒跑完就持續計算
  while not it.finished:
    # 取得目前走訪到的索引位置
    idx = it.multi_index

    # 先把原本的參數值存起來
    oldval = p[idx]

    # 將該參數往正方向微調一點
    p[idx] = oldval + h
    pos = f()       # 在參數被改動後重新呼叫 f()，取得正方向的輸出結果

    # 將該參數往負方向微調一點
    p[idx] = oldval - h
    neg = f()       # 在參數被改動後重新呼叫 f()，取得負方向的輸出結果

    # 將參數值還原成原本的狀態，避免影響下一次計算
    p[idx] = oldval

    # 使用中央差分法計算梯度，並與上游傳下來的 df 做加權
    grad[idx] = np.sum((pos - neg) * df) / (2 * h)

    # 另一種寫法（使用內積），目前被註解掉
    # grad[idx] = np.dot((pos - neg), df) / (2 * h)

    # 移動到下一個參數位置
    it.iternext()

  # 回傳整個參數 p 的數值梯度
  return grad

def f():
   return conv.forward(x)

dw_num = numerical_gradient_from_df(f,conv.W,df)
diff_error = lambda x, y: np.max(np.abs(x - y)) 
print(diff_error(conv.grads[0],dw_num))

db_num = numerical_gradient_from_df(lambda :conv.forward(x),conv.b,df)
print(diff_error(conv.grads[1],db_num))

dx_num = numerical_gradient_from_df(lambda :conv.forward(x),x,df)
print(diff_error(dx,dx_num))

5.674960501522719e-11
1.2954970429746027e-11
4.92105245442076e-11


In [32]:
class Pool(Layer):
    def __init__(self, pool_param = (2,2,2)):
        super().__init__()
        # pool_h, pool_w: 池化視窗的長與寬；stride: 步幅
        self.pool_h, self.pool_w, self.stride = pool_param

    def forward(self, x): 
        self.x = x    
        N, C, H, W = x.shape
        pool_h, pool_w, stride = self.pool_h, self.pool_w, self.stride
        
        # h_out, w_out (輸出維度)：計算公式與卷積層相似，決定輸出特徵圖的大小
        h_out = 1 + (H - pool_h) // stride
        w_out = 1 + (W - pool_w) // stride         
        out = np.zeros((N, C, h_out, w_out))
        
        # 透過四層迴圈遍歷 Batch(N)、通道(C) 與 空間位置(i, j)
        for n in range(N): # 遍歷每個輸入樣本
            for c in range(C):  # 遍歷每個通道
                for i in range(h_out): # 遍歷輸出高度位置
                    si = stride * i  # 計算對應的輸入高度位置
                    for j in range(w_out): # 遍歷輸出寬度位置
                        sj = stride * j # 計算對應的輸入寬度位置
                        # x_win (輸入視窗)：擷取目前要進行池化的局部區域
                        x_win = x[n, c, si:si+pool_h, sj:sj+pool_w]  
                        # Max Pooling：只取出視窗中的最大值作為輸出
                        out[n,c,i,j] = np.max(x_win)        
     
        return out
    
    def backward(self, dout):
        x = self.x
        N, C, H, W = x.shape
        kH, kW, stride = self.pool_h, self.pool_w, self.stride      
        oH = 1 + (H - kH) // stride
        oW = 1 + (W - kW) // stride
       
        # dx (輸入梯度)：初始化為 0，形狀與原始輸入 x 相同
        dx = np.zeros_like(x)    
  
        # dout (下層梯度)：從後方傳回對此層輸出的梯度，是計算的反向起點
        for k in range(N): # 遍歷每個輸入樣本
            for l in range(C): # 遍歷每個通道
                for i in range(oH): # 遍歷輸出高度位置
                    si = stride * i # 計算對應的輸入高度位置
                    for j in range(oW): # 遍歷輸出寬度位置
                        sj = stride * j # 計算對應的輸入寬度位置
                        # slice: 重新找出前向傳播時的視窗區域
                        slice = x[k, l, si:si+kH, sj:sj+kW]
                        slice_max = np.max(slice)
                        
                        # 梯度回傳核心：利用布林遮罩 (slice_max == slice)
                        # 只有最大值的位置會接收來自 dout 的梯度，其餘維持 0
                        dx[k, l, si:si+kH, sj:sj+kW] += (slice_max == slice) * dout[k,l,i,j] 
                    
        return dx

In [33]:
# 初始化輸入資料 x：(Batch, Channel, Height, Width) = (3, 2, 8, 8)
x = np.random.randn(3, 2, 8, 8)

# df (輸出端梯度)：模擬從下一層傳回來的梯度
# 因為 8x8 經過 2x2 且 Stride 為 2 的池化後，輸出會變成 4x4
df = np.random.randn(3, 2, 4, 4)

# 定義池化層：視窗大小 (2,2)，步幅 (Stride) 為 2
pool = Pool((2, 2, 2))

# f (前向傳播輸出)：執行池化運算
f = pool.forward(x)

# dx (解析梯度)：執行我們在 Pool 類別中寫的 backward 邏輯
dx = pool.backward(df)

# dx_num (數值梯度)：利用微小擾動（如 +0.0001 與 -0.0001）計算出的梯度
# 這是利用數學定義跑出來的「標準答案」，用來驗證 dx 是否寫錯
dx_num = numerical_gradient_from_df(lambda :pool.forward(x), x, df)

# 印出誤差比對：如果數值非常小（如 1e-10），代表你的 backward 邏輯寫對了！
print(diff_error(dx, dx_num))

1.680655614677562e-11
